# Data Collection 

This notebook 1 (out of 5) for the NLP Reddit Project
Notebook by: <b>Martijn de Vries</b><br>
martijndevries91@gmail.com <br>
https://github.com/martijndevries/NLP_Reddit <br>

<b> Note: </b> Given the 2023 Reddit API changes, PushShift API No longer works and this notebook can no longer be run to completion. The data is is still available, in the data folder of this repository.

## Problem Statement

A US political consultancy company is researching how news sources and discussed topics differ between the US political mainstream and the conservative right-wing media. In the last decade or so, the US political right-wing has been increasingly described as living in an entirely separate information ecosystem from the political mainstream. In order to gauge how intense this effect is, we will collect, process, and classify the Reddit content of two politically-themed subreddit that reflect the mainstream and conservative voters respectively: <b>r/politics</b> and <b>r/conservative</b>. 

For this project, we will build two separate branches of models: one for post submissions (largely consisting of links to news sites), and another for comments (consisting of actual Reddit users discussing political news). As this is a binary classification problem where the two classes are of equal interest and will be approximately balanced, we will use the accuracy score as the main metric to gauge the success of the classification model. 

Because political news is always evolving, we have chosen a specific moment in time: the month leading up to the 2022 midterms, October 6th to November 6th 2022. This ensures that 1) the same news cycle is covered for both subreddits, 2) both subreddits were at peak activity, and 3) maximum potential for interesting insights in the way that news is discussed within these two subreddits.

## In this Notebook

I will use the Pushshift API to obtain data from the two subreddits (r/politics and r/conservative) for the specified data. I will then extract the information I want from the obtained data, convert to a pandas dataframe and save the dataframes to two csv files: all_submissions.csv and all_comments.csv.

In [1]:
import requests
import datetime as dt
import time
import pandas as pd

## Posts

The first thing I would like to is to collect all posts made in the specified month (October 6th, 2022 until November 6th, 2022). First let's define the two subreddits:

In [6]:
subreddit1 = 'politics'
subreddit2 = 'conservative'

Now I'll write a function to actually collect data with Pushshift API. From previous experiments, it seems that I'm not always able to succesfully connect to the API. So I'll build in a couple of failsafes - I'll loop over each day of the month, and collect all the data. At the end of the loop, repeat for the days where it didn't manage to connect succesfully, until all data has been collected:

In [7]:
def collect_subreddit_data(subreddit, start_date='10-06-2022', n_days=30, rtype='submission'):
    """
    Collect submissions/comments from a given subreddit using requests and Pushshift API
    
    Parameters:
        subreddit (str): the subreddit to collect data from
        start_date (str): the starting date in mm-dd-yyyy
        n_days (int): the number of days to loop over after the start date. one requests will be made per day up to 1000 items
        rtype (str): whether to collect submissins or comments
    Returns:
        a list of json objects for each day
    """
    
    pushshift_url= 'https://api.pushshift.io/reddit/' + rtype + '/search'
    subr_data = [] #to be returned
    success_list = [] #to keep track of which days have been collected
    mm, dd, yy = start_date.split('-')
    tot_posts = 0
    
    q = 0 
    while len(success_list) < n_days: 
        
        s_epoch_start = int(dt.datetime(int(yy),int(mm),int(dd),0,0).timestamp())
        for i in range(1, n_days+1):
            
            print('-' * 30)
            if i in success_list: continue
            
            print(f'Day of the month {i}')
            s_epoch_start += 24*3600
            s_epoch_end = s_epoch_start + 24*3600
            params = {'subreddit':subreddit, 'since':s_epoch_start, 'until':s_epoch_end , 'size':1000}
            
            res = requests.get(pushshift_url, params)
            print(f'Status code: {res.status_code}')
            
            if res.status_code != 200:
                print('Connection unsuccessful')
                continue
            else:
                data = res.json()
                if len(data['data']) == 0:
                    print('Connection succesful but no data retrieved.') 
                    continue
                subr_data.append(data)
                success_list.append(i) #don't need try this day again on subsequent loops
                tot_posts += len(data['data'])
                
            time.sleep(15) #polite
        print(f'Total length of the list: {len(success_list)}')
        q +=1
        if q == 10: break # Failsafe to make sure that if the API is down, the while loop doesn't keep going forever
    print(f'Great Success! Total number of posts/comments retrieved: {tot_posts}')
    return subr_data

In [8]:
subr1_data = collect_subreddit_data(subreddit1, start_date='10-06-2022', n_days=31)

------------------------------
Day of the month 1
Status code: 200
------------------------------
Day of the month 2
Status code: 200
------------------------------
Day of the month 3
Status code: 200
------------------------------
Day of the month 4
Status code: 200
------------------------------
Day of the month 5
Status code: 200
------------------------------
Day of the month 6
Status code: 200
------------------------------
Day of the month 7
Status code: 200
------------------------------
Day of the month 8
Status code: 200
------------------------------
Day of the month 9
Status code: 200
------------------------------
Day of the month 10
Status code: 200
------------------------------
Day of the month 11
Status code: 200
------------------------------
Day of the month 12
Status code: 200
------------------------------
Day of the month 13
Status code: 200
------------------------------
Day of the month 14
Status code: 200
------------------------------
Day of the month 15
Status

In [9]:
subr2_data = collect_subreddit_data(subreddit2, start_date='10-06-2022', n_days=31)

------------------------------
Day of the month 1
Status code: 200
------------------------------
Day of the month 2
Status code: 200
------------------------------
Day of the month 3
Status code: 200
------------------------------
Day of the month 4
Status code: 200
------------------------------
Day of the month 5
Status code: 200
------------------------------
Day of the month 6
Status code: 200
------------------------------
Day of the month 7
Status code: 200
------------------------------
Day of the month 8
Status code: 200
------------------------------
Day of the month 9
Status code: 200
------------------------------
Day of the month 10
Status code: 200
------------------------------
Day of the month 11
Status code: 200
------------------------------
Day of the month 12
Status code: 200
------------------------------
Day of the month 13
Status code: 200
------------------------------
Day of the month 14
Status code: 200
------------------------------
Day of the month 15
Status

In [10]:
#display the keys of a single post dictionary
subr1_data[0]['data'][0].keys()

dict_keys(['all_awardings', 'allow_live_comments', 'archived', 'author', 'author_created_utc', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders', 'can_gild', 'category', 'content_categories', 'contest_mode', 'created_utc', 'discussion_type', 'distinguished', 'domain', 'edited', 'gilded', 'gildings', 'hidden', 'hide_score', 'id', 'is_created_from_ads_ui', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video', 'link_flair_background_color', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media', 'media_embed', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'post_hint', 'prev

subr1_data and subr2_data are now lists with the following structure:
1) each entry is all the posts for a given day
2) indexing into the lists, selecting ['data'], gives me a list with all the data for that day
3) indexing into this gives me a dictionary with the data for a single post

Let's write a function to take these lists, and return a pandas dataframe with
1) the unique ID of the post
2) the time it was posted
3) the title
4) the selftext (if present)
5) the URL the post links to
5) the number of comments
6) the upvote ratio

In [11]:
def create_submissions_df(subr_data, subr_name):
    """
    Take the posts output of collect_subreddit_data and turn it into a pandas dataframe
    """

    df_l = []
    features = ['id', 'created_utc', 'title', 'selftext', 'url', 'num_comments', 'upvote_ratio']
    for subs_day in subr_data:
        
        for post in subs_day['data']:
            post_dict = {features[i]:post[x] for i,x in enumerate(features)}
            df_l.append(post_dict)
            
    sub_df = pd.DataFrame(df_l)
    sub_df['subreddit'] = subr_name
    sub_df.set_index('id', inplace=True)
    return sub_df

Let's try this out for our first subreddit, r/politics

In [12]:
subr1_df = create_submissions_df(subr1_data, subreddit1)
print(subr1_df.shape)
subr1_df.head()

(10010, 7)

Seems like that worked!
Let's double check if there are any duplicates - there shouldn't be, since I iterated over subsequent time intervals

In [14]:
subr1_df.groupby(level=0).first().shape

(10010, 7)

Now let's repeat the procedure for r/conservative

In [35]:
subr2_df = create_submissions_df(subr2_data, subreddit2)
print(subr2_df.shape)
subr2_df.head()

(10411, 7)


,created_utc,title,selftext,url,num_comments,upvote_ratio,subreddit
id,,,,,,,
xylz2r,1665212391,Why do many conservatives consider the student...,[removed],,0,1.00,conservative
xylxh4,1665212224,Cocaine: How Albanian gangs took control of Br...,,https://www.telegraph.co.uk/world-news/2022/09...,2,0.64,conservative
xylwcf,1665212109,Ukraine: Walter Mead - The Question on Putin’s...,,https://www.wsj.com/amp/articles/would-we-risk...,8,0.55,conservative
xyltrr,1665211841,CONSERVATIVE ANSWERS ONLY,[removed],,0,1.00,conservative
xylqmu,1665211523,Iran: David Patrikarakos = This revolt is exis...,,https://www.telegraph.co.uk/news/2022/10/07/re...,2,0.70,conservative


Now let's merge this into one CSV and save

In [36]:
subr_tot = pd.concat([subr1_df, subr2_df])
#subr_tot.to_csv('../data/all_submissions.csv') #commented out to not overwrite my data

(20421, 7)

Looking at the 'selftext' columns - it seem that the vast majority of selftexts are empty, or removed/deleted:

In [33]:
subr_tot['selftext'].value_counts(normalize=True)[:3]

             0.775133
[removed]    0.179962
[deleted]    0.036482
Name: selftext, dtype: float64

In [34]:
subr_tot['selftext'].value_counts(normalize=True)[:3].sum()

0.9915772978796338

So I probably should just ignore the selftext and focus on the titles

## Comments

Secondly, I would like to look at comments to obtain more information on how people actually speak on these subreddits. Comments are not included in the submissions data, I will need to scrape them individually:

In [21]:
subr1_comments = collect_subreddit_data(subreddit1, start_date='10-06-2022', n_days=31, rtype='comment')

------------------------------
Day of the month 1
Status code: 200
------------------------------
Day of the month 2
Status code: 200
------------------------------
Day of the month 3
Status code: 200
------------------------------
Day of the month 4
Status code: 200
------------------------------
Day of the month 5
Status code: 200
------------------------------
Day of the month 6
Status code: 200
------------------------------
Day of the month 7
Status code: 200
------------------------------
Day of the month 8
Status code: 200
------------------------------
Day of the month 9
Status code: 200
------------------------------
Day of the month 10
Status code: 200
------------------------------
Day of the month 11
Status code: 200
------------------------------
Day of the month 12
Status code: 200
------------------------------
Day of the month 13
Status code: 200
------------------------------
Day of the month 14
Status code: 200
------------------------------
Day of the month 15
Status

In [22]:
subr2_comments = collect_subreddit_data(subreddit2, start_date='10-06-2022', n_days=31, rtype='comment')

------------------------------
Day of the month 1
Status code: 200
------------------------------
Day of the month 2
Status code: 200
------------------------------
Day of the month 3
Status code: 200
------------------------------
Day of the month 4
Status code: 200
------------------------------
Day of the month 5
Status code: 200
------------------------------
Day of the month 6
Status code: 524
Connection unsuccessful
------------------------------
Day of the month 7
Status code: 200
------------------------------
Day of the month 8
Status code: 200
------------------------------
Day of the month 9
Status code: 200
------------------------------
Day of the month 10
Status code: 200
------------------------------
Day of the month 11
Status code: 200
------------------------------
Day of the month 12
Status code: 200
------------------------------
Day of the month 13
Status code: 200
------------------------------
Day of the month 14
Status code: 200
------------------------------
Da

What do the keys of 'comment' dictionaries look like?

In [23]:
subr1_comments[0]['data'][0].keys()

dict_keys(['all_awardings', 'archived', 'associated_award', 'author', 'author_created_utc', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'body', 'can_gild', 'collapsed', 'collapsed_because_crowd_control', 'collapsed_reason', 'collapsed_reason_code', 'comment_type', 'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded', 'gildings', 'id', 'is_submitter', 'link_id', 'locked', 'no_follow', 'parent_id', 'permalink', 'retrieved_utc', 'score', 'score_hidden', 'send_replies', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_type', 'top_awarded_type', 'total_awards_received', 'treatment_tags', 'unrepliable_reason', 'updated_utc', 'body_sha1', 'utc_datetime_str'])

It seems like the actual comments are saved in 'body':

In [24]:
subr1_comments[0]['data'][3]['body']

'If he was making calls to Georgia for someone else, imagine what he was doing for himself in SC.'

We'll want to save these comments in another csv file. Let's write a function that saves
1) id
2) parent id 
3) the author
4) date created
5) actual comment ('body')
6) score

In [25]:
def create_comments_df(subr_data, subr_name):
    """
    Take the comments output of collect_subreddit_data and turn it into a pandas dataframe
    """

    df_l = []
    features = ['id', 'parent_id', 'author', 'created_utc', 'body', 'score']
    for subs_day in subr_data:
        for post in subs_day['data']:
            post_dict = {features[i]:post[x] for i,x in enumerate(features)}
            df_l.append(post_dict)
    com_df = pd.DataFrame(df_l)
    com_df['subreddit'] = subr_name
    com_df.set_index('id', inplace=True)
    return com_df

In [26]:
subr1_com_df = create_comments_df(subr1_comments, subreddit1)
subr1_com_df.shape

(30956, 6)

In [27]:
subr1_com_df.head()

,parent_id,author,created_utc,body,score,subreddit
id,,,,,,
irhr7g7,4.084409e+10,stickznstonez_,1665212396,https://youtu.be/i1oCQ6bZ_Ws\n\nThis guy might...,1,politics
irhr7fd,4.084099e+10,PoliticsModeratorBot,1665212395,Hi `PhilipLiptonSchrute`. [Your comment](/r/po...,1,politics
irhr7bp,NaN,valcatrina,1665212393,I am surprised it takes the FBI to draw this l...,1,politics
irhr79r,4.084347e+10,StrillyBings,1665212392,If he was making calls to Georgia for someone ...,1,politics
irhr79a,NaN,After_Ad_9636,1665212391,Duh?\n\nWhy wouldn’t he?,1,politics


Not sure about those NaN values for parent Id, but let's keep this as is.

In [28]:
subr2_com_df = create_comments_df(subr2_comments, subreddit2)
subr2_com_df.shape

(30955, 6)

In [29]:
subr2_com_df.head()

,parent_id,author,created_utc,body,score,subreddit
id,,,,,,
irhr6tz,4.084212e+10,[deleted],1665212380,[removed],1,conservative
irhr2rs,4.084264e+10,[deleted],1665212278,[removed],1,conservative
irhr1rl,NaN,Specialist861,1665212253,"This reeks of the ""Y'all will get BidenBucks i...",1,conservative
irhr1pu,4.084394e+10,WholesomeMorning,1665212252,This sub proves otherwise lol,1,conservative
irhr140,NaN,ChunkyArsenio,1665212236,&gt; **No Paywall:**\n \nhttps://archive.ph/S...,1,conservative


Now we can merge the two, and save:

In [38]:
subr_com_tot = pd.concat([subr1_com_df, subr2_com_df])
subr_com_tot.shape
#subr_com_tot.to_csv('../data/all_comments.csv') #commented out to not overwrite my data

(61911, 6)